In [1]:
import pandas as pd
import numpy as np
import keras as k
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Conv1D, Flatten, MaxPooling1D
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train_E6oV3lV.csv")

In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
data = train
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
x = tokenizer.texts_to_sequences(data['tweet'].values)
x

          id  label                                              tweet
0          1      0   user when a father is dysfunctional and is so...
1          2      0  user user thanks for lyft credit i cant use ca...
2          3      0                                bihday your majesty
3          4      0  model   i love u take with u all the time in u...
4          5      0               factsguide society now    motivation
5          6      0  [22] huge fan fare and big talking before they...
6          7      0   user camping tomorrow user user user user use...
7          8      0  the next school year is the year for exams can...
8          9      0  we won love the land allin cavs champions clev...
9         10      0         user user welcome here   im   its so gr8  
10        11      0    ireland consumer price index mom climbed fro...
11        12      0  we are so selfish orlando standwithorlando pul...
12        13      0    i get to see my daddy today   80days gettingfed
13    

[[1, 35, 4, 254, 10, 7, 10, 20, 100, 92, 252, 251, 92, 454],
 [1, 1, 170, 9, 5, 63, 433, 631, 71, 68, 1496, 8],
 [60, 25],
 [140, 5, 13, 39, 76, 16, 39, 21, 2, 41, 8, 113, 129],
 [1592, 48, 287],
 [1017, 702, 7, 261, 703, 238, 71, 362, 7, 187, 35, 71, 54, 105],
 [1, 121, 1, 1, 1, 1, 1, 1, 1],
 [2,
  150,
  258,
  166,
  10,
  2,
  166,
  9,
  1987,
  63,
  128,
  56,
  23,
  258,
  1987,
  220,
  1396,
  146],
 [33, 855, 13, 2, 1452, 1116, 1497],
 [1, 1, 727, 110, 38, 32, 20],
 [1274, 1453, 1904, 407, 44, 1063, 3, 8, 241, 133, 167, 165, 192],
 [33, 22, 20, 126, 774, 1454, 13],
 [5, 54, 3, 66, 12, 587, 47],
 [1, 1640, 1498, 1593, 258, 1905, 2, 1641, 1906],
 [64, 1397, 8, 1241, 1824],
 [10, 415],
 [5, 30, 53, 9, 249, 4, 1752, 53, 52],
 [440, 73, 6, 899],
 [32, 89, 494, 21, 266, 177, 1825, 1, 1, 84, 57],
 [50, 33, 21, 118, 1907, 22, 36, 225, 11],
 [380,
  57,
  1697,
  9,
  1698,
  55,
  17,
  1361,
  78,
  943,
  302,
  185,
  1826,
  1455,
  67,
  812,
  157,
  44,
  105],
 [123, 180, 1

In [5]:
y = train.label

In [6]:
train_y = y

In [7]:
X_train = sequence.pad_sequences(x)

In [8]:
X_train.shape

(31962, 32)

In [9]:
import keras.backend as K

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    #Computes the F score.
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [10]:
embedding_vecor_length = 128

# LSTM

In [11]:
# create the LSTM model
model = Sequential()
model.add(Embedding(max_fatures, embedding_vecor_length, input_length=X_train.shape[1]))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.summary()
model.fit(X_train, train_y, epochs=3, batch_size=128, validation_split = 0.30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 519,401
Trainable params: 519,401
Non-trainable params: 0
_________________________________________________________________
Train on 22373 samples, validate on 9589 samples
Epoch 1/3
22373/22373 [==============================] - 42s 2ms/step - loss: 0.2127 - acc: 0.9350 - val_loss: 0.1442 - val_acc: 0.9475
Epoch 2/3
22373/22373 [==============================] - 30s 1ms/step - loss: 0.1198 - acc: 0.9567 - val_loss: 0.1401 - val_acc: 0.9487
Epoch 3/3
22373/22373 [==============================] - 30s 1ms/step - loss: 0

# Predicting

In [13]:
test = pd.read_csv("test_tweets_anuFYb8.csv")

In [14]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [15]:
test_final = tokenizer.texts_to_sequences(test['tweet'].values)
X_test = sequence.pad_sequences(test_final, maxlen=32)

In [16]:
pred = model.predict(X_test)
pred1 = pd.DataFrame(pred ,columns = ['label'])

In [17]:
pred1.head()

,label
0,0.012339
1,0.052881
2,0.000341
3,0.029933
4,0.002354


In [18]:
for i, row in pred1.iterrows():
    if row['label'] > 0.5:
        row['label'] = int(1)
    else :
            row['label'] = int(0)

In [19]:
i1 = test.id
pred2 = pd.concat([i1,pred1],axis =1)

In [20]:
pred2.to_csv('pred.csv', index= False)